# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 30 2022 3:12PM,245375,16,8463087,Sartorius Lab Products and Service,Executing
1,Aug 30 2022 3:05PM,245374,22,625960,"NBTY Global, Inc.",Released
2,Aug 30 2022 2:49PM,245370,12,HH6691,Hush Hush,Released
3,Aug 30 2022 1:51PM,245352,10,8463044,Eywa Pharma Inc.,Released
4,Aug 30 2022 1:40PM,245358,19,ADV80005324,"AdvaGen Pharma, Ltd",Executing
5,Aug 30 2022 1:40PM,245358,19,ADV80005325,"AdvaGen Pharma, Ltd",Executing
6,Aug 30 2022 1:40PM,245358,19,ADV80005326,"AdvaGen Pharma, Ltd",Executing
7,Aug 30 2022 1:40PM,245358,19,ADV80005327,"AdvaGen Pharma, Ltd",Released
8,Aug 30 2022 1:40PM,245358,19,ADV80005328,"AdvaGen Pharma, Ltd",Released
9,Aug 30 2022 1:40PM,245358,19,ADV80005329,"AdvaGen Pharma, Ltd",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,245358,Executing,15
29,245358,Released,14
30,245370,Released,1
31,245374,Released,1
32,245375,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
245352,NaN,NaN,1.0
245358,NaN,15.0,14.0
245370,NaN,NaN,1.0
245374,NaN,NaN,1.0
245375,NaN,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245240,1.0,7.0,30.0
245259,26.0,35.0,3.0
245276,0.0,0.0,1.0
245277,0.0,0.0,1.0
245280,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245240,1,7,30
245259,26,35,3
245276,0,0,1
245277,0,0,1
245280,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245240,1,7,30
1,245259,26,35,3
2,245276,0,0,1
3,245277,0,0,1
4,245280,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245240,1,7,30
1,245259,26,35,3
2,245276,,,1
3,245277,,,1
4,245280,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 30 2022 3:12PM,245375,16,Sartorius Lab Products and Service
1,Aug 30 2022 3:05PM,245374,22,"NBTY Global, Inc."
2,Aug 30 2022 2:49PM,245370,12,Hush Hush
3,Aug 30 2022 1:51PM,245352,10,Eywa Pharma Inc.
4,Aug 30 2022 1:40PM,245358,19,"AdvaGen Pharma, Ltd"
33,Aug 30 2022 1:00PM,245276,21,"NBTY Global, Inc."
34,Aug 30 2022 12:59PM,245277,21,"NBTY Global, Inc."
35,Aug 30 2022 12:57PM,245280,21,"NBTY Global, Inc."
36,Aug 30 2022 12:38PM,245284,21,"NBTY Global, Inc."
37,Aug 30 2022 12:37PM,245286,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 30 2022 3:12PM,245375,16,Sartorius Lab Products and Service,,1,
1,Aug 30 2022 3:05PM,245374,22,"NBTY Global, Inc.",,,1
2,Aug 30 2022 2:49PM,245370,12,Hush Hush,,,1
3,Aug 30 2022 1:51PM,245352,10,Eywa Pharma Inc.,,,1
4,Aug 30 2022 1:40PM,245358,19,"AdvaGen Pharma, Ltd",,15,14
5,Aug 30 2022 1:00PM,245276,21,"NBTY Global, Inc.",,,1
6,Aug 30 2022 12:59PM,245277,21,"NBTY Global, Inc.",,,1
7,Aug 30 2022 12:57PM,245280,21,"NBTY Global, Inc.",,1,
8,Aug 30 2022 12:38PM,245284,21,"NBTY Global, Inc.",,,1
9,Aug 30 2022 12:37PM,245286,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 30 2022 3:12PM,245375,16,Sartorius Lab Products and Service,,1,
1,Aug 30 2022 3:05PM,245374,22,"NBTY Global, Inc.",1,,
2,Aug 30 2022 2:49PM,245370,12,Hush Hush,1,,
3,Aug 30 2022 1:51PM,245352,10,Eywa Pharma Inc.,1,,
4,Aug 30 2022 1:40PM,245358,19,"AdvaGen Pharma, Ltd",14,15,
5,Aug 30 2022 1:00PM,245276,21,"NBTY Global, Inc.",1,,
6,Aug 30 2022 12:59PM,245277,21,"NBTY Global, Inc.",1,,
7,Aug 30 2022 12:57PM,245280,21,"NBTY Global, Inc.",,1,
8,Aug 30 2022 12:38PM,245284,21,"NBTY Global, Inc.",1,,
9,Aug 30 2022 12:37PM,245286,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 30 2022 3:12PM,245375,16,Sartorius Lab Products and Service,,1,
1,Aug 30 2022 3:05PM,245374,22,"NBTY Global, Inc.",1,,
2,Aug 30 2022 2:49PM,245370,12,Hush Hush,1,,
3,Aug 30 2022 1:51PM,245352,10,Eywa Pharma Inc.,1,,
4,Aug 30 2022 1:40PM,245358,19,"AdvaGen Pharma, Ltd",14,15,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 30 2022 3:12PM,245375,16,Sartorius Lab Products and Service,NaN,1.0,NaN
1,Aug 30 2022 3:05PM,245374,22,"NBTY Global, Inc.",1.0,NaN,NaN
2,Aug 30 2022 2:49PM,245370,12,Hush Hush,1.0,NaN,NaN
3,Aug 30 2022 1:51PM,245352,10,Eywa Pharma Inc.,1.0,NaN,NaN
4,Aug 30 2022 1:40PM,245358,19,"AdvaGen Pharma, Ltd",14.0,15.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 30 2022 3:12PM,245375,16,Sartorius Lab Products and Service,0.0,1.0,0.0
1,Aug 30 2022 3:05PM,245374,22,"NBTY Global, Inc.",1.0,0.0,0.0
2,Aug 30 2022 2:49PM,245370,12,Hush Hush,1.0,0.0,0.0
3,Aug 30 2022 1:51PM,245352,10,Eywa Pharma Inc.,1.0,0.0,0.0
4,Aug 30 2022 1:40PM,245358,19,"AdvaGen Pharma, Ltd",14.0,15.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,981259,5.0,0.0,2.0
12,245370,1.0,0.0,0.0
15,490595,15.0,35.0,60.0
16,245375,0.0,1.0,0.0
19,490701,14.0,16.0,0.0
20,981211,98.0,24.0,1.0
21,2207547,7.0,2.0,0.0
22,490721,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,981259,5.0,0.0,2.0
1,12,245370,1.0,0.0,0.0
2,15,490595,15.0,35.0,60.0
3,16,245375,0.0,1.0,0.0
4,19,490701,14.0,16.0,0.0
5,20,981211,98.0,24.0,1.0
6,21,2207547,7.0,2.0,0.0
7,22,490721,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,5.0,0.0,2.0
1,12,1.0,0.0,0.0
2,15,15.0,35.0,60.0
3,16,0.0,1.0,0.0
4,19,14.0,16.0,0.0
5,20,98.0,24.0,1.0
6,21,7.0,2.0,0.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,5.0
1,12,Released,1.0
2,15,Released,15.0
3,16,Released,0.0
4,19,Released,14.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,2.0,0.0,60.0,0.0,0.0,1.0,0.0,0.0
Executing,0.0,0.0,35.0,1.0,16.0,24.0,2.0,0.0
Released,5.0,1.0,15.0,0.0,14.0,98.0,7.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,2.0,0.0,60.0,0.0,0.0,1.0,0.0,0.0
1,Executing,0.0,0.0,35.0,1.0,16.0,24.0,2.0,0.0
2,Released,5.0,1.0,15.0,0.0,14.0,98.0,7.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,2.0,0.0,60.0,0.0,0.0,1.0,0.0,0.0
1,Executing,0.0,0.0,35.0,1.0,16.0,24.0,2.0,0.0
2,Released,5.0,1.0,15.0,0.0,14.0,98.0,7.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()